### Theory of bed profile forms implemented in some numpy functions
A bed profile model is required to estimate the bathymetry of a channel in 2D. Here we identify a set of functions to estimate the depth profile from a set of parameters that can be measured iun-situ or (ideally!) only at the side of the channel from drone footage. The angles at the sides of the channel are, according to Savenije 2003 and referenced authors, a good representation of bed form similarities. We therefore take these assumed relationships as starting point for our assessment.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Replication of figure 1 in Savenije 2003

In [ ]:
# let's start with some coordinates from left (0) to right bank

def depth_from_width(s, phi=0.1*np.pi, h_m=10):
    """
    Simplest implementation of Eq. 12 from Savenije 2003, where the first coordinate (zero meters) is the lowest point. We stop at zero as that
    is where we reach the natural levee crest.
    """
    # zero is the lowest point in the stream starting in the middle of the stream
    angle = np.minimum(np.tan(phi)/h_m * s, 0.5*np.pi)
    depth = -np.cos(angle)*h_m
    return depth


s = np.linspace(0, 100, 1000)

depth = depth_from_width(s)

plt.plot(s, depth)


## Implementation over an entire symmetrical channel

Now we assume that the left coordinate is at the natural levee's left bank and the right at the natural levee's right bank.
We can now no longer use both `h_m` and `phi`. We have to resolve one of them to be able to solve the problem.

In [ ]:
def depth_profile_from_parameters(s, phi=None, h_m=None):
    """
    Solving the entire profile using a symmetrical assumption and either phi or h_m defined
    """
    if phi is None and h_m is None:
        raise ValueError("Either phi or h_m has to be supplied. You supplied nothing.")
    if phi is not None and h_m is not None:
        raise ValueError("Either phi or h_m has to be supplied. You supplied both. Select either one of the two")
    # compute the total width of the channel
    B = s[-1] - s[0]
    if phi is None:
        # h_m was supplied, so resolve for phi
        phi = np.arctan(np.pi*h_m/B)
    else:
        # phi was supplied, so resolve for h_m
        h_m = B*np.tan(phi)/np.pi
    # compute ordinate coordinates (i.e. from center line towards banks)
    _s = s - 0.5*B
    return depth_from_width(_s, phi=phi, h_m=h_m)
    
        

## Test over a set of natural angles of repose
Let's see what happens with different angles of repose. We should be assuming that milder angles lead to shallower water

In [ ]:

for n, phi in enumerate(np.linspace(0.05*np.pi, 0.25*np.pi, 10)):
    depth = depth_profile_from_parameters(s, phi=phi)
    plt.plot(s, depth, label="{:02d}: phi = {:1.2f}".format(n + 1, phi))
    plt.xlabel("distance (left to right bank) [m]")
    plt.ylabel("depth [m]")
plt.legend()

In [ ]:
depth_profile_from_parameters(s, h_m=31.831)

### Asymmetrical channel behaviour
A channel bed is usually not entirely symmetrical, so we can also assume that some further constraints may be necessary to assess the width/depth relationship. If we supply the left and right angle, then this does not necessarily mean that the depth can be resolved from that, as they may to a certain degree counter. Hence we have to optimize the location over the cross section where the depth is at maximum.



In [ ]:
#TODO